In [1]:
import pandas as pd
import json

# Leer el JSON
with open("full-data-limpia-remax.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convertir a DataFrame
df = pd.DataFrame(data)



In [ ]:
#df = df.drop(columns=["error", "raw"], errors="ignore")

In [ ]:
#df = df.loc[:, df.notna().sum() > 10]

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1314 entries, 0 to 1313
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   titulo       1314 non-null   object
 1   operacion    1314 non-null   object
 2   direccion    1314 non-null   object
 3   Precio       1314 non-null   object
 4   Ambientes    1314 non-null   object
 5   Baños        1314 non-null   object
 6   Total        1314 non-null   object
 7   Cubierta     1314 non-null   object
 8   Descubierta  1314 non-null   object
 9   Balcón       1314 non-null   object
 10  Antigüedad   1314 non-null   object
 11  Disposición  1314 non-null   object
 12  https        1314 non-null   object
dtypes: object(13)
memory usage: 133.6+ KB


In [3]:
df["direccion"] = df["direccion"].str.split(",").str[0].str.strip()

In [4]:
df.rename(columns={
    "titulo": "Titulo",
    "operacion": "Operacion",
    "direccion": "Direccion",
    "https": "Inmobiliaria"
}, inplace=True)

In [ ]:
#df.drop(columns=["http"], inplace=True)

In [5]:
df['Barrio'] = "Palermo"

In [29]:
"""
# Separar ubicacion por coma
df[['barrio', 'ciudad']] = df['ubicacion'].str.split(',', expand=True)[[1, 2]]

# Limpiar espacios
df['barrio'] = df['barrio'].str.strip()
df['ciudad'] = df['ciudad'].str.strip()
df = df.drop(columns=["ubicacion"])
"""

'\n# Separar ubicacion por coma\ndf[[\'barrio\', \'ciudad\']] = df[\'ubicacion\'].str.split(\',\', expand=True)[[1, 2]]\n\n# Limpiar espacios\ndf[\'barrio\'] = df[\'barrio\'].str.strip()\ndf[\'ciudad\'] = df[\'ciudad\'].str.strip()\ndf = df.drop(columns=["ubicacion"])\n'

In [30]:
"""
df['fecha_publicacion'] = df['fecha_publicacion'].str.extract(r'(\d{1,2}/\d{1,2}/\d{4})')
df['fecha_publicacion'] = pd.to_datetime(df['fecha_publicacion'], dayfirst=True)
"""

"\ndf['fecha_publicacion'] = df['fecha_publicacion'].str.extract(r'(\\d{1,2}/\\d{1,2}/\\d{4})')\ndf['fecha_publicacion'] = pd.to_datetime(df['fecha_publicacion'], dayfirst=True)\n"

In [6]:
import re

def limpiar_direccion(direccion):
    if pd.isna(direccion):
        return direccion
    
    # Convertir a string por si acaso
    direccion = str(direccion)
    
    # 1. Eliminar todo lo que está entre paréntesis (incluyendo los paréntesis)
    direccion = re.sub(r'\([^)]*\)', '', direccion)
    
    # 2. Eliminar lo que está después de ciertos separadores
    direccion = direccion.split(' -')[0]  # después de guión
    direccion = direccion.split(' y')[0]  # después de 'y'
    direccion = direccion.split(',')[0]   # después de coma
    
    # 3. Correcciones específicas conocidas
    calle_reemplazos = {
        r'\bOro\b': 'Fray Justo Santa María de Oro',
        r'\bSta\.?\b': 'Santa',
        r'\bSta María\b': 'Santa María',
        r"\b(Al\s+){2,}": " ",
        r"\b(al\s+){2,}": " ",
    }
    
    for patron, reemplazo in calle_reemplazos.items():
        direccion = re.sub(patron, reemplazo, direccion, flags=re.IGNORECASE)
    
    # 4. Limpieza final - eliminar espacios múltiples y trim
    direccion = re.sub(r'\s+', ' ', direccion).strip()
    
    return direccion

In [11]:
# Aplicar la función a tu DataFrame
df['Direccion'] = df['Direccion'].apply(limpiar_direccion)

df['Direccion'] = df['Direccion'].str.upper()

# Mostrar algunos ejemplos
print(df[['Direccion', 'Direccion']].head())

                         Direccion                        Direccion
0                     CORDOBA 4027                     CORDOBA 4027
1                    SANTA FE 5268                    SANTA FE 5268
2                     CÓRDOBA 4309                     CÓRDOBA 4309
3  SOLDADO DE LA INDEPENDENCIA 704  SOLDADO DE LA INDEPENDENCIA 704
4                       CABILDO 20                       CABILDO 20


In [12]:
#1
df["Direccion"] = df["Direccion"].str.replace(r"\s*al\s+", " ", regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'Entre\s+[^0-9]*\s+[Yy]\s+[^0-9]*', " ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)\b(av\.?|avenida)\b'," ",regex=True)
df['Direccion'] = df['Direccion'].str.replace('.', '')
df['Direccion'] = df['Direccion'].str.replace(',', '')
df['Direccion'] = df['Direccion'].str.replace(r'(?i)\s+y\s+[^-\d]+'," ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*piso\s*\d+'," ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*unidad\s*\w+'," ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*cfrt'," ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*uf\s*\d+', " ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*\d+\s*amb', " ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*\d+°?º?p', " ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*frt', " ",regex=True) 
df['Direccion'] = df['Direccion'].str.replace(r'(?i)-\s*\d+°?\s*piso', " ",regex=True)
df['Direccion'] = df['Direccion'].str.replace(r',\s*C?\d{4}.*', '', regex=True)


In [13]:
df

,Titulo,Operacion,Direccion,Precio,Ambientes,Baños,Total,Cubierta,Descubierta,Balcón,Antigüedad,Disposición,Inmobiliaria,Barrio
0,Emprendimiento en construcción entre 1 y 3 amb...,Mirá este emprendimiento en RE/MAX: UrbanHaus ...,CORDOBA 4027,72.501USD,3.0,2.0,0,0,0,0,0,0,https://www.remax.com.ar/emprendimientos/urban...,Palermo
1,Emprendimiento en terminado entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: Departamen...,SANTA FE 5268,65.875USD,2.0,1.0,0,0,0,0,0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo
2,Emprendimiento en pre venta entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: ¡Descubre ...,CÓRDOBA 4309,82.000USD,2.0,1.0,36.00 m²,31.00 m²,5.00 m²,0,0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo
3,Emprendimiento en pre venta entre 1 y 4 ambien...,Mirá este emprendimiento en RE/MAX: Cacique De...,SOLDADO DE LA INDEPENDENCIA 704,83.000USD,4.0,3.0,0,0,0,0,0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo
4,Emprendimiento en construcción entre 2 y 4 amb...,Mirá este emprendimiento en RE/MAX: VENTA PALE...,CABILDO 20,62.900USD,4.0,2.0,0,0,0,0,0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,Departamento en venta 4 ambientes en Avenida G...,Mirá esta propiedad en RE/MAX: VENTA DE DEPART...,GENER LAS HERAS 3800,379.000USD,4.0,3.0,150.00 m²,150.00 m²,0.00 m²,0,56.0 años,0,https://www.remax.com.ar/listings/vta-dpto-pal...,Palermo
1310,Departamento en venta 1 ambiente en Carranza 1...,Mirá esta propiedad en RE/MAX: COMIENZO DE OBR...,CARRANZA 1400,117.000USD,1.0,1.0,34.06 m²,29.29 m²,4.77 m²,0,1.0 años,0,https://www.remax.com.ar/listings/vta-monoambi...,Palermo
1311,Departamento en venta 2 ambientes en Güemes 45...,Mirá esta propiedad en RE/MAX: Amplios departa...,GÜEMES 4500,180.000USD,2.0,1.0,54.79 m²,52.26 m²,2.53 m²,0,1.0 años,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo
1312,Departamento en venta 1 ambiente en Niceto Veg...,Mirá esta propiedad en RE/MAX: Lanzamiento de ...,NICETO VEGA 5100,98.000USD,1.0,1.0,32.00 m²,28.00 m²,4.00 m²,0,2.0 años,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo


In [14]:
def limpiar_columnas_numericas(df, columnas):
    for columna in columnas:
        if columna in df.columns:
            # Extrae solo los números, incluyendo puntos
            df[columna] = df[columna].astype(str).str.extract(r'(\d[\d\.]*)')
            # Convierte a tipo numérico
            df[columna] = pd.to_numeric(df[columna], errors='coerce')
    return df

In [16]:
df = limpiar_columnas_numericas(df, ["Precio", "Expensas", "Total", "Cubierta","Ambientes", "Baños", "Antigüedad", "Dormitorios"])
df

,Titulo,Operacion,Direccion,Precio,Ambientes,Baños,Total,Cubierta,Descubierta,Balcón,Antigüedad,Disposición,Inmobiliaria,Barrio
0,Emprendimiento en construcción entre 1 y 3 amb...,Mirá este emprendimiento en RE/MAX: UrbanHaus ...,CORDOBA 4027,72.501,3.0,2.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/urban...,Palermo
1,Emprendimiento en terminado entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: Departamen...,SANTA FE 5268,65.875,2.0,1.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo
2,Emprendimiento en pre venta entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: ¡Descubre ...,CÓRDOBA 4309,82.000,2.0,1.0,36.00,31.00,5.00 m²,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo
3,Emprendimiento en pre venta entre 1 y 4 ambien...,Mirá este emprendimiento en RE/MAX: Cacique De...,SOLDADO DE LA INDEPENDENCIA 704,83.000,4.0,3.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo
4,Emprendimiento en construcción entre 2 y 4 amb...,Mirá este emprendimiento en RE/MAX: VENTA PALE...,CABILDO 20,62.900,4.0,2.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,Departamento en venta 4 ambientes en Avenida G...,Mirá esta propiedad en RE/MAX: VENTA DE DEPART...,GENER LAS HERAS 3800,379.000,4.0,3.0,150.00,150.00,0.00 m²,0,56.0,0,https://www.remax.com.ar/listings/vta-dpto-pal...,Palermo
1310,Departamento en venta 1 ambiente en Carranza 1...,Mirá esta propiedad en RE/MAX: COMIENZO DE OBR...,CARRANZA 1400,117.000,1.0,1.0,34.06,29.29,4.77 m²,0,1.0,0,https://www.remax.com.ar/listings/vta-monoambi...,Palermo
1311,Departamento en venta 2 ambientes en Güemes 45...,Mirá esta propiedad en RE/MAX: Amplios departa...,GÜEMES 4500,180.000,2.0,1.0,54.79,52.26,2.53 m²,0,1.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo
1312,Departamento en venta 1 ambiente en Niceto Veg...,Mirá esta propiedad en RE/MAX: Lanzamiento de ...,NICETO VEGA 5100,98.000,1.0,1.0,32.00,28.00,4.00 m²,0,2.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo


In [17]:
def categorizar_antiguedad(antiguedad_str):
    try:
        # Extraer solo los números (incluyendo decimales)
        antiguedad_num = float(re.sub(r'[^\d.]', '', str(antiguedad_str)))
    except:
        return None
    
    # Redondear al entero más cercano
    antiguedad = int(round(antiguedad_num))
    
    if antiguedad == 0:
        return 0  # A estrenar
    elif antiguedad <= 5:
        return 1  # Muy nueva
    elif antiguedad <= 10:
        return 2  # Nueva
    elif antiguedad <= 20:
        return 3  # Relativamente nueva
    elif antiguedad <= 40:
        return 4  # Media
    else:
        return 5  # Antigua

# Aplicar la función
df["Categoria_antigüedad"] = df["Antigüedad"].apply(categorizar_antiguedad)

df

,Titulo,Operacion,Direccion,Precio,Ambientes,Baños,Total,Cubierta,Descubierta,Balcón,Antigüedad,Disposición,Inmobiliaria,Barrio,Categoria_antigüedad
0,Emprendimiento en construcción entre 1 y 3 amb...,Mirá este emprendimiento en RE/MAX: UrbanHaus ...,CORDOBA 4027,72.501,3.0,2.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/urban...,Palermo,0
1,Emprendimiento en terminado entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: Departamen...,SANTA FE 5268,65.875,2.0,1.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0
2,Emprendimiento en pre venta entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: ¡Descubre ...,CÓRDOBA 4309,82.000,2.0,1.0,36.00,31.00,5.00 m²,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0
3,Emprendimiento en pre venta entre 1 y 4 ambien...,Mirá este emprendimiento en RE/MAX: Cacique De...,SOLDADO DE LA INDEPENDENCIA 704,83.000,4.0,3.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0
4,Emprendimiento en construcción entre 2 y 4 amb...,Mirá este emprendimiento en RE/MAX: VENTA PALE...,CABILDO 20,62.900,4.0,2.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,Departamento en venta 4 ambientes en Avenida G...,Mirá esta propiedad en RE/MAX: VENTA DE DEPART...,GENER LAS HERAS 3800,379.000,4.0,3.0,150.00,150.00,0.00 m²,0,56.0,0,https://www.remax.com.ar/listings/vta-dpto-pal...,Palermo,5
1310,Departamento en venta 1 ambiente en Carranza 1...,Mirá esta propiedad en RE/MAX: COMIENZO DE OBR...,CARRANZA 1400,117.000,1.0,1.0,34.06,29.29,4.77 m²,0,1.0,0,https://www.remax.com.ar/listings/vta-monoambi...,Palermo,1
1311,Departamento en venta 2 ambientes en Güemes 45...,Mirá esta propiedad en RE/MAX: Amplios departa...,GÜEMES 4500,180.000,2.0,1.0,54.79,52.26,2.53 m²,0,1.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1
1312,Departamento en venta 1 ambiente en Niceto Veg...,Mirá esta propiedad en RE/MAX: Lanzamiento de ...,NICETO VEGA 5100,98.000,1.0,1.0,32.00,28.00,4.00 m²,0,2.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1


In [18]:
df['Publicada_en']="Remax"

df

,Titulo,Operacion,Direccion,Precio,Ambientes,Baños,Total,Cubierta,Descubierta,Balcón,Antigüedad,Disposición,Inmobiliaria,Barrio,Categoria_antigüedad,Publicada_en
0,Emprendimiento en construcción entre 1 y 3 amb...,Mirá este emprendimiento en RE/MAX: UrbanHaus ...,CORDOBA 4027,72.501,3.0,2.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/urban...,Palermo,0,Remax
1,Emprendimiento en terminado entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: Departamen...,SANTA FE 5268,65.875,2.0,1.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax
2,Emprendimiento en pre venta entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: ¡Descubre ...,CÓRDOBA 4309,82.000,2.0,1.0,36.00,31.00,5.00 m²,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax
3,Emprendimiento en pre venta entre 1 y 4 ambien...,Mirá este emprendimiento en RE/MAX: Cacique De...,SOLDADO DE LA INDEPENDENCIA 704,83.000,4.0,3.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax
4,Emprendimiento en construcción entre 2 y 4 amb...,Mirá este emprendimiento en RE/MAX: VENTA PALE...,CABILDO 20,62.900,4.0,2.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,Departamento en venta 4 ambientes en Avenida G...,Mirá esta propiedad en RE/MAX: VENTA DE DEPART...,GENER LAS HERAS 3800,379.000,4.0,3.0,150.00,150.00,0.00 m²,0,56.0,0,https://www.remax.com.ar/listings/vta-dpto-pal...,Palermo,5,Remax
1310,Departamento en venta 1 ambiente en Carranza 1...,Mirá esta propiedad en RE/MAX: COMIENZO DE OBR...,CARRANZA 1400,117.000,1.0,1.0,34.06,29.29,4.77 m²,0,1.0,0,https://www.remax.com.ar/listings/vta-monoambi...,Palermo,1,Remax
1311,Departamento en venta 2 ambientes en Güemes 45...,Mirá esta propiedad en RE/MAX: Amplios departa...,GÜEMES 4500,180.000,2.0,1.0,54.79,52.26,2.53 m²,0,1.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1,Remax
1312,Departamento en venta 1 ambiente en Niceto Veg...,Mirá esta propiedad en RE/MAX: Lanzamiento de ...,NICETO VEGA 5100,98.000,1.0,1.0,32.00,28.00,4.00 m²,0,2.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1,Remax


In [20]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from functools import lru_cache
from tqdm import tqdm
import logging
logging.getLogger("geopy").setLevel(logging.ERROR)

def geocodificar_direcciones(df, columna_direccion="Direccion", columna_barrio="Barrio"):
    geolocator = Nominatim(user_agent="cabaprop_tasador")
    rate_limited_geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

    @lru_cache(maxsize=10000)
    def geocode_cached(direccion):
        try:
            return rate_limited_geocode(direccion)
        except Exception as e:
            print(f"Error al geocodificar '{direccion}': {e}")
            return None

    latitudes = []
    longitudes = []

    tqdm.pandas(desc="Geocodificando")

    for direccion_completa in tqdm(
        df.apply(lambda row: f"{row[columna_direccion]}, {row[columna_barrio]}, CABA, Argentina", axis=1),
        total=len(df),
        desc="Procesando direcciones"
    ):
        location = geocode_cached(direccion_completa)
        if location:
            latitudes.append(location.latitude)
            longitudes.append(location.longitude)
        else:
            latitudes.append(None)
            longitudes.append(None)

    df["lat"] = latitudes
    df["lon"] = longitudes

    return df

In [21]:
geocodificar_direcciones(df)

Procesando direcciones: 100%|██████████| 1314/1314 [15:20<00:00,  1.43it/s] 


,Titulo,Operacion,Direccion,Precio,Ambientes,Baños,Total,Cubierta,Descubierta,Balcón,Antigüedad,Disposición,Inmobiliaria,Barrio,Categoria_antigüedad,Publicada_en,lat,lon
0,Emprendimiento en construcción entre 1 y 3 amb...,Mirá este emprendimiento en RE/MAX: UrbanHaus ...,CORDOBA 4027,72.501,3.0,2.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/urban...,Palermo,0,Remax,-34.594282,-58.429894
1,Emprendimiento en terminado entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: Departamen...,SANTA FE 5268,65.875,2.0,1.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.575847,-58.433892
2,Emprendimiento en pre venta entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: ¡Descubre ...,CÓRDOBA 4309,82.000,2.0,1.0,36.00,31.00,5.00 m²,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.594282,-58.429894
3,Emprendimiento en pre venta entre 1 y 4 ambien...,Mirá este emprendimiento en RE/MAX: Cacique De...,SOLDADO DE LA INDEPENDENCIA 704,83.000,4.0,3.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.567935,-58.433937
4,Emprendimiento en construcción entre 2 y 4 amb...,Mirá este emprendimiento en RE/MAX: VENTA PALE...,CABILDO 20,62.900,4.0,2.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.574895,-58.435972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,Departamento en venta 4 ambientes en Avenida G...,Mirá esta propiedad en RE/MAX: VENTA DE DEPART...,GENER LAS HERAS 3800,379.000,4.0,3.0,150.00,150.00,0.00 m²,0,56.0,0,https://www.remax.com.ar/listings/vta-dpto-pal...,Palermo,5,Remax,NaN,NaN
1310,Departamento en venta 1 ambiente en Carranza 1...,Mirá esta propiedad en RE/MAX: COMIENZO DE OBR...,CARRANZA 1400,117.000,1.0,1.0,34.06,29.29,4.77 m²,0,1.0,0,https://www.remax.com.ar/listings/vta-monoambi...,Palermo,1,Remax,-34.575518,-58.434715
1311,Departamento en venta 2 ambientes en Güemes 45...,Mirá esta propiedad en RE/MAX: Amplios departa...,GÜEMES 4500,180.000,2.0,1.0,54.79,52.26,2.53 m²,0,1.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1,Remax,-34.582288,-58.422843
1312,Departamento en venta 1 ambiente en Niceto Veg...,Mirá esta propiedad en RE/MAX: Lanzamiento de ...,NICETO VEGA 5100,98.000,1.0,1.0,32.00,28.00,4.00 m²,0,2.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1,Remax,-34.590361,-58.434277


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1314 entries, 0 to 1313
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Titulo                1314 non-null   object 
 1   Operacion             1314 non-null   object 
 2   Direccion             1314 non-null   object 
 3   Precio                1295 non-null   float64
 4   Ambientes             1314 non-null   float64
 5   Baños                 1314 non-null   float64
 6   Total                 1314 non-null   float64
 7   Cubierta              1314 non-null   float64
 8   Descubierta           1314 non-null   object 
 9   Balcón                1314 non-null   object 
 10  Antigüedad            1314 non-null   float64
 11  Disposición           1314 non-null   object 
 12  Inmobiliaria          1314 non-null   object 
 13  Barrio                1314 non-null   object 
 14  Categoria_antigüedad  1314 non-null   int64  
 15  Publicada_en         

In [ ]:
"""import numpy as np

# Opción A: Reemplazar TODOS los tipos de ceros por NaN
df['Total'] = df['Total'].replace([0, '0', '0.0', '0,0', 0.0, '0 m²'], np.nan)

# Opción B: Eliminar filas donde Total es cualquier tipo de cero
df = df[~df['Total'].isin([0, '0', '0.0', '0,0', 0.0, '0 m²'])]
df = df.reset_index(drop=True)

# Opción C: Convertir a numérico primero (si hay mezcla de tipos)
df['Total'] = pd.to_numeric(df['Total'].astype(str).str.replace(' m²', '').str.replace(',', '.'), errors='coerce')""""""
df['Total'] = df['Total'].replace(0, np.nan)

In [23]:
print("\nDespués de la limpieza:")
print("Valores únicos en 'Total':", df['Total'].unique())
print("Cantidad de NaN:", df['Total'].isna().sum())
print("\nPrimeras filas:")
print(df[['Titulo', 'Total']].head())


Después de la limpieza:
Valores únicos en 'Total': [  0.    36.    54.    42.28  37.26  34.    34.34 113.16  56.    27.47
  65.12  57.33  54.61  62.    44.36 122.34  59.37  31.74  42.    28.8
  33.    48.44  54.35  58.09  57.54  43.    84.   118.13 122.18 142.18
  72.36  53.33  50.81  61.    62.72  58.    50.3   76.    89.97  44.
  60.    87.   104.86 201.94 113.44 126.41 215.    84.32  86.    65.
  23.61  41.7   36.42  27.    64.89  47.7   72.    47.9   75.73  29.45
  37.5   50.    40.    49.16  38.61  64.    62.5   37.27  39.4   63.6
  33.17  46.38  39.93  37.61  44.13  43.47  35.    47.02  52.    43.1
  39.5   46.42  65.06 133.   126.16  55.03  69.7   59.    58.55  92.
  71.45  48.1   50.92  83.49  61.25  81.89  80.01  82.42 108.   103.54
 102.   145.   154.   134.    78.89 153.5  109.97 149.71  92.85 125.
 128.   173.3  178.    97.8  106.   126.78 115.5   75.45 183.    97.
 234.   243.   184.72 265.72 240.45 224.4  230.   373.93 227.33  38.3
 101.3   45.48  71.1   66.37 127.13  31

In [36]:
# Eliminar filas donde Precio, lon o lat sean NaN
df_clean = df.dropna(subset=['Precio', 'Total', 'lon', 'lat'], how='any')

In [24]:
print("Valores únicos en 'Total' después de la limpieza:")
print(df['Total'].value_counts(dropna=False))

print("\nCantidad de ceros restantes:", (df['Total'] == 0).sum())

Valores únicos en 'Total' después de la limpieza:
Total
32.00     18
35.00     16
34.00     15
36.00     15
37.00     14
          ..
49.44      1
52.90      1
64.85      1
265.54     1
36.08      1
Name: count, Length: 868, dtype: int64

Cantidad de ceros restantes: 5


In [37]:
df

,Titulo,Operacion,Direccion,Precio,Ambientes,Baños,Total,Cubierta,Descubierta,Balcón,Antigüedad,Disposición,Inmobiliaria,Barrio,Categoria_antigüedad,Publicada_en,lat,lon
0,Emprendimiento en construcción entre 1 y 3 amb...,Mirá este emprendimiento en RE/MAX: UrbanHaus ...,CORDOBA 4027,72.501,3.0,2.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/urban...,Palermo,0,Remax,-34.594282,-58.429894
1,Emprendimiento en terminado entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: Departamen...,SANTA FE 5268,65.875,2.0,1.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.575847,-58.433892
2,Emprendimiento en pre venta entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: ¡Descubre ...,CÓRDOBA 4309,82.000,2.0,1.0,36.00,31.00,5.00 m²,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.594282,-58.429894
3,Emprendimiento en pre venta entre 1 y 4 ambien...,Mirá este emprendimiento en RE/MAX: Cacique De...,SOLDADO DE LA INDEPENDENCIA 704,83.000,4.0,3.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.567935,-58.433937
4,Emprendimiento en construcción entre 2 y 4 amb...,Mirá este emprendimiento en RE/MAX: VENTA PALE...,CABILDO 20,62.900,4.0,2.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.574895,-58.435972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309,Departamento en venta 4 ambientes en Avenida G...,Mirá esta propiedad en RE/MAX: VENTA DE DEPART...,GENER LAS HERAS 3800,379.000,4.0,3.0,150.00,150.00,0.00 m²,0,56.0,0,https://www.remax.com.ar/listings/vta-dpto-pal...,Palermo,5,Remax,NaN,NaN
1310,Departamento en venta 1 ambiente en Carranza 1...,Mirá esta propiedad en RE/MAX: COMIENZO DE OBR...,CARRANZA 1400,117.000,1.0,1.0,34.06,29.29,4.77 m²,0,1.0,0,https://www.remax.com.ar/listings/vta-monoambi...,Palermo,1,Remax,-34.575518,-58.434715
1311,Departamento en venta 2 ambientes en Güemes 45...,Mirá esta propiedad en RE/MAX: Amplios departa...,GÜEMES 4500,180.000,2.0,1.0,54.79,52.26,2.53 m²,0,1.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1,Remax,-34.582288,-58.422843
1312,Departamento en venta 1 ambiente en Niceto Veg...,Mirá esta propiedad en RE/MAX: Lanzamiento de ...,NICETO VEGA 5100,98.000,1.0,1.0,32.00,28.00,4.00 m²,0,2.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1,Remax,-34.590361,-58.434277


In [79]:
df_clean

,Titulo,Operacion,Direccion,Precio,Ambientes,Baños,Total,Cubierta,Descubierta,Balcón,Antigüedad,Disposición,Inmobiliaria,Barrio,Categoria_antigüedad,Publicada_en,lat,lon
0,Emprendimiento en construcción entre 1 y 3 amb...,Mirá este emprendimiento en RE/MAX: UrbanHaus ...,CORDOBA 4027,72.501,3.0,2.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/urban...,Palermo,0,Remax,-34.594282,-58.429894
1,Emprendimiento en terminado entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: Departamen...,SANTA FE 5268,65.875,2.0,1.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.575847,-58.433892
2,Emprendimiento en pre venta entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: ¡Descubre ...,CÓRDOBA 4309,82.000,2.0,1.0,36.00,31.00,5.00 m²,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.594282,-58.429894
3,Emprendimiento en pre venta entre 1 y 4 ambien...,Mirá este emprendimiento en RE/MAX: Cacique De...,SOLDADO DE LA INDEPENDENCIA 704,83.000,4.0,3.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.567935,-58.433937
4,Emprendimiento en construcción entre 2 y 4 amb...,Mirá este emprendimiento en RE/MAX: VENTA PALE...,CABILDO 20,62.900,4.0,2.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.574895,-58.435972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,Departamento semipiso en venta 4 ambientes en ...,Mirá esta propiedad en RE/MAX: El Equipo Remax...,REPÚBLICA DE LA INDIA 3000,379.900,4.0,3.0,117.00,110.00,7.00 m²,0,41.0,0,https://www.remax.com.ar/listings/vta-dpto-4-a...,Palermo,5,Remax,-34.580431,-58.416188
1310,Departamento en venta 1 ambiente en Carranza 1...,Mirá esta propiedad en RE/MAX: COMIENZO DE OBR...,CARRANZA 1400,117.000,1.0,1.0,34.06,29.29,4.77 m²,0,1.0,0,https://www.remax.com.ar/listings/vta-monoambi...,Palermo,1,Remax,-34.575518,-58.434715
1311,Departamento en venta 2 ambientes en Güemes 45...,Mirá esta propiedad en RE/MAX: Amplios departa...,GÜEMES 4500,180.000,2.0,1.0,54.79,52.26,2.53 m²,0,1.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1,Remax,-34.582288,-58.422843
1312,Departamento en venta 1 ambiente en Niceto Veg...,Mirá esta propiedad en RE/MAX: Lanzamiento de ...,NICETO VEGA 5100,98.000,1.0,1.0,32.00,28.00,4.00 m²,0,2.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1,Remax,-34.590361,-58.434277


In [80]:
conteo_ceros = df_clean['Total'].value_counts().get(0, 0)
print(f"Cantidad de registros con total = 0: {conteo_ceros}")

Cantidad de registros con total = 0: 5


In [81]:
nan_precio = df_clean["Precio"].isna().sum()
print(f"Número de propiedades sin precio: {nan_precio}")

nan_Total = df_clean["Total"].isna().sum()
print(f"Número de propiedades sin Total: {nan_Total}")

Número de propiedades sin precio: 0
Número de propiedades sin Total: 0


In [82]:
def limpiar_columnas_numericas(df, columnas):
    for columna in columnas:
        if columna in df.columns:
            df[columna] = (
                df[columna]
                .astype(str)
                .str.replace(r'[^\d.,]', '', regex=True)  # dejar números, comas, puntos
                .str.replace(',', '.', regex=False)       # estandarizar decimales
            )
            # Quitar punto si actúa como separador de miles (solo si hay más de un punto)
            df[columna] = df[columna].str.replace(r'(?<=\d)\.(?=\d{3}(?:\.|$))', '', regex=True)
            df[columna] = pd.to_numeric(df[columna], errors='coerce')
    return df

In [83]:
copia = df_clean.copy()

In [84]:
copia

,Titulo,Operacion,Direccion,Precio,Ambientes,Baños,Total,Cubierta,Descubierta,Balcón,Antigüedad,Disposición,Inmobiliaria,Barrio,Categoria_antigüedad,Publicada_en,lat,lon
0,Emprendimiento en construcción entre 1 y 3 amb...,Mirá este emprendimiento en RE/MAX: UrbanHaus ...,CORDOBA 4027,72.501,3.0,2.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/urban...,Palermo,0,Remax,-34.594282,-58.429894
1,Emprendimiento en terminado entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: Departamen...,SANTA FE 5268,65.875,2.0,1.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.575847,-58.433892
2,Emprendimiento en pre venta entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: ¡Descubre ...,CÓRDOBA 4309,82.000,2.0,1.0,36.00,31.00,5.00 m²,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.594282,-58.429894
3,Emprendimiento en pre venta entre 1 y 4 ambien...,Mirá este emprendimiento en RE/MAX: Cacique De...,SOLDADO DE LA INDEPENDENCIA 704,83.000,4.0,3.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.567935,-58.433937
4,Emprendimiento en construcción entre 2 y 4 amb...,Mirá este emprendimiento en RE/MAX: VENTA PALE...,CABILDO 20,62.900,4.0,2.0,0.00,0.00,0,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.574895,-58.435972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,Departamento semipiso en venta 4 ambientes en ...,Mirá esta propiedad en RE/MAX: El Equipo Remax...,REPÚBLICA DE LA INDIA 3000,379.900,4.0,3.0,117.00,110.00,7.00 m²,0,41.0,0,https://www.remax.com.ar/listings/vta-dpto-4-a...,Palermo,5,Remax,-34.580431,-58.416188
1310,Departamento en venta 1 ambiente en Carranza 1...,Mirá esta propiedad en RE/MAX: COMIENZO DE OBR...,CARRANZA 1400,117.000,1.0,1.0,34.06,29.29,4.77 m²,0,1.0,0,https://www.remax.com.ar/listings/vta-monoambi...,Palermo,1,Remax,-34.575518,-58.434715
1311,Departamento en venta 2 ambientes en Güemes 45...,Mirá esta propiedad en RE/MAX: Amplios departa...,GÜEMES 4500,180.000,2.0,1.0,54.79,52.26,2.53 m²,0,1.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1,Remax,-34.582288,-58.422843
1312,Departamento en venta 1 ambiente en Niceto Veg...,Mirá esta propiedad en RE/MAX: Lanzamiento de ...,NICETO VEGA 5100,98.000,1.0,1.0,32.00,28.00,4.00 m²,0,2.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1,Remax,-34.590361,-58.434277


In [85]:
copia = limpiar_columnas_numericas(copia, ["Total", "Cubierta","Descubierta","Ambientes", "Baños", "Antigüedad", "Dormitorios"])

copia

,Titulo,Operacion,Direccion,Precio,Ambientes,Baños,Total,Cubierta,Descubierta,Balcón,Antigüedad,Disposición,Inmobiliaria,Barrio,Categoria_antigüedad,Publicada_en,lat,lon
0,Emprendimiento en construcción entre 1 y 3 amb...,Mirá este emprendimiento en RE/MAX: UrbanHaus ...,CORDOBA 4027,72.501,3.0,2.0,0.00,0.00,0.00,0,0.0,0,https://www.remax.com.ar/emprendimientos/urban...,Palermo,0,Remax,-34.594282,-58.429894
1,Emprendimiento en terminado entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: Departamen...,SANTA FE 5268,65.875,2.0,1.0,0.00,0.00,0.00,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.575847,-58.433892
2,Emprendimiento en pre venta entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: ¡Descubre ...,CÓRDOBA 4309,82.000,2.0,1.0,36.00,31.00,5.00,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.594282,-58.429894
3,Emprendimiento en pre venta entre 1 y 4 ambien...,Mirá este emprendimiento en RE/MAX: Cacique De...,SOLDADO DE LA INDEPENDENCIA 704,83.000,4.0,3.0,0.00,0.00,0.00,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.567935,-58.433937
4,Emprendimiento en construcción entre 2 y 4 amb...,Mirá este emprendimiento en RE/MAX: VENTA PALE...,CABILDO 20,62.900,4.0,2.0,0.00,0.00,0.00,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.574895,-58.435972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,Departamento semipiso en venta 4 ambientes en ...,Mirá esta propiedad en RE/MAX: El Equipo Remax...,REPÚBLICA DE LA INDIA 3000,379.900,4.0,3.0,117.00,110.00,7.00,0,41.0,0,https://www.remax.com.ar/listings/vta-dpto-4-a...,Palermo,5,Remax,-34.580431,-58.416188
1310,Departamento en venta 1 ambiente en Carranza 1...,Mirá esta propiedad en RE/MAX: COMIENZO DE OBR...,CARRANZA 1400,117.000,1.0,1.0,34.06,29.29,4.77,0,1.0,0,https://www.remax.com.ar/listings/vta-monoambi...,Palermo,1,Remax,-34.575518,-58.434715
1311,Departamento en venta 2 ambientes en Güemes 45...,Mirá esta propiedad en RE/MAX: Amplios departa...,GÜEMES 4500,180.000,2.0,1.0,54.79,52.26,2.53,0,1.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1,Remax,-34.582288,-58.422843
1312,Departamento en venta 1 ambiente en Niceto Veg...,Mirá esta propiedad en RE/MAX: Lanzamiento de ...,NICETO VEGA 5100,98.000,1.0,1.0,32.00,28.00,4.00,0,2.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1,Remax,-34.590361,-58.434277


In [86]:
copia["Precio"]

0        72.501
1        65.875
2        82.000
3        83.000
4        62.900
         ...   
1308    379.900
1310    117.000
1311    180.000
1312     98.000
1313    115.000
Name: Precio, Length: 1120, dtype: float64

In [87]:
# Solución correcta para tu caso
copia["Precio"] = (copia["Precio"] * 1000).astype(int)

copia["Precio"]

0        72501
1        65875
2        82000
3        83000
4        62900
         ...  
1308    379900
1310    117000
1311    180000
1312     98000
1313    115000
Name: Precio, Length: 1120, dtype: int64

In [88]:

copia


,Titulo,Operacion,Direccion,Precio,Ambientes,Baños,Total,Cubierta,Descubierta,Balcón,Antigüedad,Disposición,Inmobiliaria,Barrio,Categoria_antigüedad,Publicada_en,lat,lon
0,Emprendimiento en construcción entre 1 y 3 amb...,Mirá este emprendimiento en RE/MAX: UrbanHaus ...,CORDOBA 4027,72501,3.0,2.0,0.00,0.00,0.00,0,0.0,0,https://www.remax.com.ar/emprendimientos/urban...,Palermo,0,Remax,-34.594282,-58.429894
1,Emprendimiento en terminado entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: Departamen...,SANTA FE 5268,65875,2.0,1.0,0.00,0.00,0.00,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.575847,-58.433892
2,Emprendimiento en pre venta entre 1 y 2 ambien...,Mirá este emprendimiento en RE/MAX: ¡Descubre ...,CÓRDOBA 4309,82000,2.0,1.0,36.00,31.00,5.00,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.594282,-58.429894
3,Emprendimiento en pre venta entre 1 y 4 ambien...,Mirá este emprendimiento en RE/MAX: Cacique De...,SOLDADO DE LA INDEPENDENCIA 704,83000,4.0,3.0,0.00,0.00,0.00,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.567935,-58.433937
4,Emprendimiento en construcción entre 2 y 4 amb...,Mirá este emprendimiento en RE/MAX: VENTA PALE...,CABILDO 20,62900,4.0,2.0,0.00,0.00,0.00,0,0.0,0,https://www.remax.com.ar/emprendimientos/venta...,Palermo,0,Remax,-34.574895,-58.435972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1308,Departamento semipiso en venta 4 ambientes en ...,Mirá esta propiedad en RE/MAX: El Equipo Remax...,REPÚBLICA DE LA INDIA 3000,379900,4.0,3.0,117.00,110.00,7.00,0,41.0,0,https://www.remax.com.ar/listings/vta-dpto-4-a...,Palermo,5,Remax,-34.580431,-58.416188
1310,Departamento en venta 1 ambiente en Carranza 1...,Mirá esta propiedad en RE/MAX: COMIENZO DE OBR...,CARRANZA 1400,117000,1.0,1.0,34.06,29.29,4.77,0,1.0,0,https://www.remax.com.ar/listings/vta-monoambi...,Palermo,1,Remax,-34.575518,-58.434715
1311,Departamento en venta 2 ambientes en Güemes 45...,Mirá esta propiedad en RE/MAX: Amplios departa...,GÜEMES 4500,180000,2.0,1.0,54.79,52.26,2.53,0,1.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1,Remax,-34.582288,-58.422843
1312,Departamento en venta 1 ambiente en Niceto Veg...,Mirá esta propiedad en RE/MAX: Lanzamiento de ...,NICETO VEGA 5100,98000,1.0,1.0,32.00,28.00,4.00,0,2.0,0,https://www.remax.com.ar/listings/vta-palermo-...,Palermo,1,Remax,-34.590361,-58.434277


In [92]:

sorted(copia["Precio"].unique())


[0,
 1000,
 4000,
 45000,
 53000,
 54000,
 54900,
 55000,
 58000,
 59000,
 60000,
 62000,
 62900,
 63900,
 64000,
 65000,
 65875,
 66000,
 67000,
 69000,
 69500,
 69900,
 70000,
 71000,
 71312,
 71500,
 72000,
 72501,
 73000,
 74000,
 74500,
 74900,
 74999,
 75000,
 75030,
 76000,
 77000,
 78000,
 79000,
 79500,
 79900,
 80000,
 81900,
 82000,
 83000,
 84000,
 84500,
 85000,
 86000,
 86800,
 87000,
 87500,
 88000,
 89000,
 89900,
 90000,
 91500,
 91900,
 92000,
 93000,
 94000,
 94900,
 95000,
 95900,
 96000,
 97000,
 97500,
 98000,
 99000,
 99700,
 99900,
 100000,
 100900,
 102000,
 104000,
 104040,
 104800,
 105000,
 105900,
 106000,
 107000,
 108000,
 109000,
 109500,
 109700,
 109900,
 110000,
 111000,
 111200,
 112000,
 114000,
 115000,
 115200,
 116000,
 117000,
 118000,
 118400,
 118900,
 119000,
 119900,
 120000,
 120360,
 122000,
 122364,
 122500,
 122900,
 123000,
 124000,
 125000,
 126000,
 127000,
 127500,
 128000,
 128500,
 129000,
 129900,
 130000,
 130900,
 132000,
 13300

In [89]:
copia.to_json("datos_geolocalizados_remax.json", orient="records", force_ascii=False)